# Analítica Web - Cálculo de Métricas
#### Alba Márquez Rodríguez

A continuación se presenta la actividad *Cálculo de Métricas*

## DATASET: LogsDataiku.csv

Este dataset almacena información de acceso al sitio web de Dataiku, durante 2 meses. Contiene información sobre cada vista de página individual en el sitio web www.Dataiku.com

## Estructura del dataset
El dataset, está en un fichero csv con 18 campos:
- server_ts: fecha de conexión al servidor
- client_ts: fecha de conexión del usuario
- client_addr: dirección IP del usuario
- visitor_id: id asignado al usuario (~ _utma)
- session_id: id de la sesión/visita (~ _utmb)
- location: url de la página a la que se accede
- referer: url de la página desde la que se accedió a location. Si está vacío se considera una búsqueda directa
- user_agent: navegador desde el que se conectó el usuario
- type: tipo de acceso
- visitor_param
- sesion_param:
- event_param:
- br_width: ancho del navegador del usuario
- br_height: alto del navegador del usuario
- sc_width: ancho de la pantalla del usuario (resolución)
- sc_height: alto de la pantalla del usuario (resolución)
- br_lan: idioma del navegador del usuario
- tz_off: time zona. El número indica la diferencia en minutos con la hora GMT . -60 indica zona horaria GMT+1 y 60 indica zona horaria GMT-1

## Ejercicio
A partir del dataset anterior, calcula las siguientes métricas para el periodo de tiempo registrado
- N.º de visitas → Número de visitas (sesiones que ha tenido el sitio web)
- N.º de visitantes únicos → Número de usuarios diferentes que han visitado el sitio web
- N.º medio de páginas/visitas →Para cada visita (sesión) cuántas páginas se han visitado. Media para todas las visitas
- Tasa de rebote → Número de visitas (sesiones) que solo han accedido a una página
- Tasa de salida para cada página → % de veces que cada página ha sido una página de salida
- Tráfico directo → Número de visitas (sesiones) que provienen de escribir la url directamente
- Tráfico de búsqueda → Número de visitas (sesiones) que provienen de pinchar en una búsqueda
- Tráfico referido → Número de visitas (sesiones) que provienen de pinchar en un enlace de otra página


#### Librerías

In [4]:
import pandas as pd
import numpy as np

#### Carga de los datos

Para comenzar cargaremos los datos desde el csv

In [5]:
# Cargar dataset
df = pd.read_csv('LogsDataiku/LogsDataiku.csv')

### Número de visitas (sesiones que ha tenido el sitio web)

Este código calcula el número de visitas únicas al sitio web. Para hacerlo, se selecciona la columna 'session_id', que contiene los identificadores de sesión únicos para cada visita al sitio web. Luego, se aplica el método nunique() que devuelve el número de valores únicos en esa columna, es decir, el número total de visitas únicas.

In [6]:
num_visits = df['session_id'].nunique()
print("Número de visitas:", num_visits)

Número de visitas: 3946


El *número de visitas* se refiere al total de sesiones que han tenido lugar en el sitio web durante el período de tiempo analizado. En este caso, el resultado indica que se han registrado un total de 3946 sesiones o visitas al sitio web durante el período de tiempo considerado. Cada vez que un usuario accede al sitio web se cuenta como una visita, independientemente de cuántas páginas visite o de la duración de la sesión.

### Número de visitantes únicos

Se extrae la columna 'visitor_id' del DataFrame df, que contiene identificadores únicos para cada visitante del sitio web. Luego, se utiliza el método nunique() para contar el número de valores únicos en esa columna, lo que da como resultado el número total de personas visitantes únicas al sitio web.

In [7]:
num_unique_visitors = df['visitor_id'].nunique()
print("Número de visitantes únicos:", num_unique_visitors)

Número de visitantes únicos: 2537


El *número de visitantes únicos* representa la cantidad total de personas usuarias individuales que han visitado el sitio web durante el período de tiempo analizado. En este contexto, el resultado de 2537 indica que durante el período registrado, 2537 personas usuarias únicas accedieron al sitio web al menos una vez.

### Número medio de páginas/visitas

Aquí se agrupa el DataFrame por el 'session_id' utilizando el método groupby(). Luego, se calcula el tamaño de cada grupo (es decir, el número de páginas visitadas en cada sesión) usando el método size(). Finalmente, se calcula la media de estos tamaños de grupo utilizando el método mean(), lo que nos da el número medio de páginas por visita.

In [8]:
avg_pages_per_visit = df.groupby('session_id').size().mean()
print("Número medio de páginas/visitas:", avg_pages_per_visit)

Número medio de páginas/visitas: 2.749366447034972



El *número medio de páginas por visita* indica la cantidad promedio de páginas que una persona usuaria ve durante una sesión en el sitio web. En este caso, el resultado de aproximadamente 2.75 significa que, en promedio, durante cada visita al sitio web, las personas usuarias navegan por alrededor de 2.75 páginas.

### Tasa de rebote

En esta parte del código, el DataFrame se agrupa por el 'session_id' con groupby(). Luego, se calcula el tamaño de cada grupo, que representa el número de páginas visitadas en cada sesión, utilizando el método size(). Después, se compara si el tamaño de cada grupo es igual a 1, lo que indica que la sesión consiste en una sola página vista. Esto devuelve una serie de booleanos donde True representa una tasa de rebote (sesión de una sola página) y False representa una sesión con más de una página visitada. Finalmente, se calcula la media de esta serie de booleanos utilizando el método mean(), lo que nos da la proporción de sesiones que resultaron en una tasa de rebote.

In [9]:
bounce_rate = (df.groupby('session_id').size() == 1).mean()
print("Tasa de rebote:", bounce_rate)

Tasa de rebote: 0.5486568677141409


La *tasa de rebote* se refiere al porcentaje de visitas que se realizaron en una interacción de una sola página, es decir, aquellas visitas en las que una persona usuaria accedió a una sola página del sitio web y luego abandonó el sitio sin interactuar más. En este caso, el resultado de aproximadamente 0.55, es decir, el 55%, indica que alrededor del 55% de las visitas al sitio web resultaron en una interacción de una sola página.

### Tasa de salida para cada página

Aquí, se filtran las filas del DataFrame donde la columna 'referer' es nula, lo que indica que es una página de salida. Luego, se agrupa este subconjunto de datos por la ubicación de la página ('location') utilizando el método groupby(). Se calcula el tamaño de cada grupo, que representa la cantidad de veces que cada página ha sido una página de salida, utilizando el método size(). Esto se divide por el tamaño de cada grupo del DataFrame original, que representa el número total de veces que cada página ha sido visitada. El resultado es una serie que contiene la tasa de salida para cada página.

In [10]:
exit_pages = df[df['referer'].isna()].groupby('location').size() / df.groupby('location').size()
print("Tasa de salida para cada página:")
print(exit_pages)

Tasa de salida para cada página:
location
http://dataiku.com/                             0.483791
http://dataiku.com/applications/                0.006803
http://dataiku.com/applications/advertising/         NaN
http://dataiku.com/applications/ecommerce/      0.020833
http://dataiku.com/applications/freemium/       0.019608
                                                  ...   
http://www.dataiku.com/products/pricing/        0.050473
http://www.dataiku.com/products/shaker/         0.013245
http://www.dataiku.com/products/sharing/        0.016304
http://www.dataiku.com/products/thankyou/       0.730769
http://www.dataiku.com/products/trynow/         0.008929
Length: 96, dtype: float64


La *tasa de salida para cada página* indica el porcentaje de veces que cada página del sitio web ha sido la última página visitada en una sesión antes de que la persona usuaria abandone el sitio.

### Tráfico directo

En este fragmento, se seleccionan las filas del DataFrame donde la columna 'referer' es nula, lo que indica que el tráfico proviene de un acceso directo (es decir, el usuario escribió la URL directamente en la barra de direcciones del navegador). Luego, se seleccionan los identificadores únicos de sesión ('session_id') de estas filas utilizando la notación de acceso a columnas. Finalmente, se calcula el número de sesiones únicas utilizando el método nunique(), lo que nos da el número total de visitas únicas que provienen de tráfico directo.

In [11]:
direct_traffic = df[df['referer'].isna()]['session_id'].nunique()
print("Tráfico directo:", direct_traffic)

Tráfico directo: 1098


El *tráfico directo* se refiere al número total de visitas al sitio web que se originaron a partir de personas usuarias que escribieron la URL directamente en la barra de direcciones del navegador, en lugar de llegar al sitio a través de un enlace de referencia o un motor de búsqueda. En este caso, el resultado de 1098 indica que hubo 1098 visitas al sitio web que fueron directamente a través de tráfico directo.

### Tráfico de búsqueda

Aquí, se seleccionan las filas del DataFrame donde la columna 'referer' contiene referencias a motores de búsqueda como Google, Bing o Yahoo. Esto se hace mediante el método str.contains() que busca patrones de texto que coincidan con las expresiones regulares 'google', 'bing' o 'yahoo', sin importar si están en mayúsculas o minúsculas (case=False). Además, na=False se utiliza para que los valores nulos no generen un error en la búsqueda. Luego, se seleccionan los identificadores únicos de sesión ('session_id') de estas filas utilizando la notación de acceso a columnas. Finalmente, se calcula el número de sesiones únicas utilizando el método nunique(), lo que nos da el número total de visitas únicas que provienen de tráfico de búsqueda.

In [12]:
search_traffic = df[df['referer'].str.contains('google|bing|yahoo', case=False, na=False)]['session_id'].nunique()
print("Tráfico de búsqueda:", search_traffic)

Tráfico de búsqueda: 1448


El *tráfico de búsqueda* se refiere al número total de visitas al sitio web que se originaron a partir de los resultados de búsqueda en buscadores, en este caso se han cogido Google, Bing o Yahoo para realizar la búsqueda. El resultado de 1448 indica que hubo 1448 visitas al sitio web que fueron generadas a través de resultados de búsqueda en alguno de estos buscadores.

### Tráfico referido

En este fragmento, se seleccionan las filas del DataFrame donde la columna 'referer' no es nula, lo que indica que el tráfico proviene de un enlace de referencia de otra página web. Se utiliza la condición notna() para seleccionar solo aquellas filas donde 'referer' tiene un valor válido. Luego, se seleccionan los identificadores únicos de sesión ('session_id') de estas filas utilizando la notación de acceso a columnas. Finalmente, se calcula el número de sesiones únicas utilizando el método nunique(), lo que nos da el número total de visitas únicas que provienen de tráfico referido.

In [13]:
referral_traffic = df[df['referer'].notna()]['session_id'].nunique()
print("Tráfico referido:", referral_traffic)

Tráfico referido: 3244


El *tráfico referido* se refiere al número total de visitas al sitio web que se originaron a partir de enlaces de referencia desde otras páginas web. En este caso, el resultado de 3244 indica que hubo 3244 visitas al sitio web que fueron dirigidas desde enlaces en otras páginas web.